In [ ]:
# Text preprocessing of parallel data for baseline training corpus

In [3]:
import pandas as pd
import seaborn as sns
import glob
import re
import math
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
!pip install nltk
from nltk.tokenize import word_tokenize

# get all files based on language

de_files = glob.glob('bar-de/cv/*de[1-5]')
bar_files = glob.glob('bar-de/cv/*bar[1-5]')

print(bar_files)
print(de_files)

['bar-de/cv/bar4', 'bar-de/cv/bar3', 'bar-de/cv/bar1', 'bar-de/cv/bar2', 'bar-de/cv/bar5']
['bar-de/cv/de1', 'bar-de/cv/de5', 'bar-de/cv/de2', 'bar-de/cv/de3', 'bar-de/cv/de4']


In [5]:
# reorder parallel files

bar_files_reordered = sorted(bar_files)
de_files_reordered = sorted(de_files)

print(bar_files_reordered)
print(de_files_reordered)

['bar-de/cv/bar1', 'bar-de/cv/bar2', 'bar-de/cv/bar3', 'bar-de/cv/bar4', 'bar-de/cv/bar5']
['bar-de/cv/de1', 'bar-de/cv/de2', 'bar-de/cv/de3', 'bar-de/cv/de4', 'bar-de/cv/de5']


In [6]:
# read files into list as lines

bar_lines = []

for file in bar_files_reordered:
    with open(file) as f:
        for line in f.readlines():
            bar_lines.append(line.rstrip(" \n"))
            
de_lines = []

for file in de_files_reordered:
    with open(file) as f:
        for line in f.readlines():
            de_lines.append(line.rstrip(" \n"))           

In [7]:
# check no. of lines

print("Bavarian Parallel Lines", len(bar_lines))
print("German Parallel Lines: ", len(de_lines))

Bavarian Parallel Lines 42266
German Parallel Lines:  42266


In [8]:
print(bar_lines[100:105])
print(de_lines[100:105])

['jahrbuch 1970 .', '2012 hods bei dancing stars gwunnan , seit 2014 is jurorin bei da vuiksvablödungs-show die .', 'de drei gemarkunga im waldaschaffer forst trong de noma waldaschaff , mespelbrunn und .', 'in da zeit vom napoleon is bayern am ofang auf da seitn vo frankreich gstandn und hos so .', 'die aus den usa stommende religiöse bewegung scientology hot do an von vier .']
['jahrbuch 1970 .', 'mit sidos 2011 begonnener jurorentätigkeit in der österreichischen orf-talentshow die .', 'die drei gemarkungen im waldaschaffer forst tragen die namen waldaschaff , mespelbrunn und .', 'zur zeit napoleons stand bayern anfangs auf der seite frankreichs und konnte durch .', 'die aus den usa stammende religiöse bewegung scientology hat hier einen ihrer vier .']


In [9]:
# transform lists of lines into dataframe

df = pd.DataFrame(bar_lines, columns = ["bar"])
df["de"] = de_lines
df.head()

,bar,de
0,da espace middllandis a grossregion in da schw...,der espace mittelland ist eine grossregion in ...
1,juli 2009 bis zum 10 .,2009 bis dez .
2,"band 50 , iberoamericana/vervuert , frankfurt ...","band 50. iberoamericana/vervuert , frankfurt a..."
3,johann jakob wicks nachrichtensammlung .,johann jakob wicks nachrichtensammlung .
4,"wagenbach , berlin 2001 .","wagenbach , berlin 2001 ."


In [15]:
# add the back-translated texts

# for bar-de BT (de is original and bar is translted)
de_BT_sampled_file = "bar-de/mono/BT-de-sampled.txt"
bar_BT_translated_file = "bar-de/mono/BT-bar-translated.txt"

de_BT_sampled_lines = []

with open(de_BT_sampled_file) as f:
    for line in f.readlines():
        de_BT_sampled_lines.append(line.rstrip(" \n"))
            
bar_BT_translated_lines = []

with open(bar_BT_translated_file) as f:
    for line in f.readlines():
        bar_BT_translated_lines.append(line.rstrip(" \n"))
        
de_BT_sampled_lines.remove('*****') # remove the BT indicator 
        
print(len(de_BT_sampled_lines))
print(len(bar_BT_translated_lines))

10000
10000


In [16]:
bar_de_df = pd.DataFrame(bar_BT_translated_lines, columns = ["bar"])
bar_de_df["de"] = de_BT_sampled_lines
bar_de_df.head()

,bar,de
0,"auf computer.howstuffworks.com , obgruafa am 5 .","auf computer.howstuffworks.com , abgerufen am 5 ."
1,rote rosen2009 .,rote rosen2009 .
2,das klima is a monsunklima .,das klima ist ein monsunklima .
3,im bremer landtag san de grünen gegenwärtig mi...,im bremer landtag sind die grünen gegenwärtig ...
4,"august 2016 , obgruafa am 6 .","18. august 2016 , abgerufen am 6 ."


In [19]:
# concat original parallel and BT texts to one dataframe

frames = [bar_de_df, df]
bar_de_all_df = pd.concat(frames)
bar_de_all_df

,bar,de
0,"auf computer.howstuffworks.com , obgruafa am 5 .","auf computer.howstuffworks.com , abgerufen am 5 ."
1,rote rosen2009 .,rote rosen2009 .
2,das klima is a monsunklima .,das klima ist ein monsunklima .
3,im bremer landtag san de grünen gegenwärtig mi...,im bremer landtag sind die grünen gegenwärtig ...
4,"august 2016 , obgruafa am 6 .","18. august 2016 , abgerufen am 6 ."
...,...,...
42261,pforrkiach san giuliano grindt .,pfarrkirche san giuliano .
42262,schneebericht des anninger-rodelvereines mödli...,schneebericht des anninger-rodelvereines mödli...
42263,picassos spätwerk .,picassos spätwerk .
42264,eva heilmann .,clara schumann .


In [22]:
bar_de_all_df = bar_de_all_df.sample(frac = 1).reset_index(drop = True)
print(len(bar_de_all_df))
bar_de_all_df.head()

52266


,bar,de
0,rosa karl .,carl .
1,de bwl bschäftigt se mit de ökonomischn funkti...,ag aufgegangen und wird heute unter dem dach d...
2,"verlag courier dover publications , 2002 , aus...","verlag courier dover publications , 2002 , aus..."
3,die simpsons und die mythen der gesellschaft .,die simpsons und die mythen der gesellschaft .
4,vo hakodate und tomakomai gibt 's a fährvabind...,von hakodate und tomakomai besteht eine fährve...


In [23]:
bar_de_all_df1, bar_de_all_df2, bar_de_all_df3, bar_de_all_df4, bar_de_all_df5 = np.array_split(bar_de_all_df, 5)

In [25]:
print("bar_de_all_df1 length: ", len(bar_de_all_df1))
print("bar_de_all_df2 length: ", len(bar_de_all_df2))
print("bar_de_all_df3 length: ", len(bar_de_all_df3))
print("bar_de_all_df4 length: ", len(bar_de_all_df4))
print("bar_de_all_df5 length: ", len(bar_de_all_df5))

bar_de_all_df1 length:  10454
bar_de_all_df2 length:  10453
bar_de_all_df3 length:  10453
bar_de_all_df4 length:  10453
bar_de_all_df5 length:  10453


In [35]:
with open("bar-de/cv_BT/bar-de/bar1", "w") as file:
    for line in bar_de_all_df1["bar"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/bar-de/bar2", "w") as file:
    for line in bar_de_all_df2["bar"]:
        file.write(line + "\n")

with open("bar-de/cv_BT/bar-de/bar3", "w") as file:
    for line in bar_de_all_df3["bar"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/bar-de/bar4", "w") as file:
    for line in bar_de_all_df4["bar"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/bar-de/bar5", "w") as file:
    for line in bar_de_all_df5["bar"]:
        file.write(line + "\n")

# de
with open("bar-de/cv_BT/bar-de/de1", "w") as file:
    for line in bar_de_all_df1["de"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/bar-de/de2", "w") as file:
    for line in bar_de_all_df2["de"]:
        file.write(line + "\n")

with open("bar-de/cv_BT/bar-de/de3", "w") as file:
    for line in bar_de_all_df3["de"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/bar-de/de4", "w") as file:
    for line in bar_de_all_df4["de"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/bar-de/de5", "w") as file:
    for line in bar_de_all_df5["de"]:
        file.write(line + "\n")

In [17]:
# for de-bar BT
bar_BT_sampled_file = "de-bar/mono/BT-bar-sampled.txt"
de_BT_translated_file = "de-bar/mono/BT-de-translated.txt"

bar_BT_sampled_lines = []

with open(bar_BT_sampled_file) as f:
    for line in f.readlines():
        bar_BT_sampled_lines.append(line.rstrip(" \n"))
            
de_BT_translated_lines = []

with open(de_BT_translated_file) as f:
    for line in f.readlines():
        de_BT_translated_lines.append(line.rstrip(" \n"))
        
bar_BT_sampled_lines.remove('*****') # remove the BT indicator 
        
print(len(bar_BT_sampled_lines))
print(len(de_BT_translated_lines))

10000
10000


In [18]:
de_bar_df = pd.DataFrame(bar_BT_sampled_lines, columns = ["bar"])
de_bar_df["de"] = de_BT_translated_lines
de_bar_df.head()

,bar,de
0,ois dag wiad heitzdog gern a topfndag gnumma .,als dag wird heitztag gern ein topfendag genum...
1,da gscheite woa ollawäu da zwickl .,der gescheite war allerwäu der zwickl .
2,"genau so wia de englenda heit butterfly song ,...",genau so wie die englender heit butterfly song...
3,heit san de sioux in vui vaschiedane reservate...,heute sind die sioux in vielen verschiedene re...
4,"da bauhnhof wean südtirolerplotz is des , wos ...","der bauhnhof werden südtirolerplatz ist das , ..."


In [21]:
frames1 = [de_bar_df, df]
de_bar_all_df = pd.concat(frames1)
de_bar_all_df

,bar,de
0,ois dag wiad heitzdog gern a topfndag gnumma .,als dag wird heitztag gern ein topfendag genum...
1,da gscheite woa ollawäu da zwickl .,der gescheite war allerwäu der zwickl .
2,"genau so wia de englenda heit butterfly song ,...",genau so wie die englender heit butterfly song...
3,heit san de sioux in vui vaschiedane reservate...,heute sind die sioux in vielen verschiedene re...
4,"da bauhnhof wean südtirolerplotz is des , wos ...","der bauhnhof werden südtirolerplatz ist das , ..."
...,...,...
42261,pforrkiach san giuliano grindt .,pfarrkirche san giuliano .
42262,schneebericht des anninger-rodelvereines mödli...,schneebericht des anninger-rodelvereines mödli...
42263,picassos spätwerk .,picassos spätwerk .
42264,eva heilmann .,clara schumann .


In [27]:
de_bar_all_df = de_bar_all_df.sample(frac = 1).reset_index(drop = True)
print(len(de_bar_all_df))
de_bar_all_df.head()

52266


,bar,de
0,aicurzio grenzt unmiddlboa an de provinz lecco .,aicurzio grenzt unmittelbar an die provinz lec...
1,de visueje rexx-entwicklungsumgebung vx-rexx h...,die visueje rex-entwicklungsumgebung vx-rexx h...
2,unmiddlboa sidli liegt da 567 m houche wolfsbü...,unmittelbar südlich liegt der 567 m hohe wolfs...
3,"sie umfossn in oan nördlichn , östlichn und we...","sie umfassen in einem nördlichen , östlichen u..."
4,eine regionale spezialität .,eine regionale spezialität .


In [29]:
de_bar_all_df1, de_bar_all_df2, de_bar_all_df3, de_bar_all_df4, de_bar_all_df5 = np.array_split(de_bar_all_df, 5)

In [30]:
print("de_bar_all_df1 length: ", len(de_bar_all_df1))
print("de_bar_all_df2 length: ", len(de_bar_all_df2))
print("de_bar_all_df3 length: ", len(de_bar_all_df3))
print("de_bar_all_df4 length: ", len(de_bar_all_df4))
print("de_bar_all_df5 length: ", len(de_bar_all_df5))

de_bar_all_df1 length:  10454
de_bar_all_df2 length:  10453
de_bar_all_df3 length:  10453
de_bar_all_df4 length:  10453
de_bar_all_df5 length:  10453


In [36]:
with open("bar-de/cv_BT/de-bar/bar1", "w") as file:
    for line in de_bar_all_df1["bar"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/de-bar/bar2", "w") as file:
    for line in de_bar_all_df2["bar"]:
        file.write(line + "\n")

with open("bar-de/cv_BT/de-bar/bar3", "w") as file:
    for line in de_bar_all_df3["bar"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/de-bar/bar4", "w") as file:
    for line in de_bar_all_df4["bar"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/de-bar/bar5", "w") as file:
    for line in de_bar_all_df5["bar"]:
        file.write(line + "\n")

# de
with open("bar-de/cv_BT/de-bar/de1", "w") as file:
    for line in de_bar_all_df1["de"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/de-bar/de2", "w") as file:
    for line in de_bar_all_df2["de"]:
        file.write(line + "\n")

with open("bar-de/cv_BT/de-bar/de3", "w") as file:
    for line in de_bar_all_df3["de"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/de-bar/de4", "w") as file:
    for line in de_bar_all_df4["de"]:
        file.write(line + "\n")
        
with open("bar-de/cv_BT/de-bar/de5", "w") as file:
    for line in de_bar_all_df5["de"]:
        file.write(line + "\n")